# How to submit to Quantinuum H-series Devices?


In [1]:
import pytket
import pytket.extensions.quantinuum


print("pytket version: ", pytket.__version__)
print("pytket-extensions-quantinuum version: ", pytket.extensions.quantinuum.__extension_version__)

pytket version:  1.25.0
pytket-extensions-quantinuum version:  0.30.0


## Circuit Preparation

In [2]:
from pytket.circuit import Circuit, fresh_symbol
from pytket.circuit.display import render_circuit_jupyter


# set up a bell test
circuit = Circuit(2, name="Bell State Test")
circuit.H(0) # Hadamard gate
circuit.CX(0, 1) # CNOT gate
circuit.measure_all() # measure all qubits

# render the circuit
render_circuit_jupyter(circuit)

## Select Device

In [3]:
from pytket.extensions.quantinuum import QuantinuumBackend

machine = 'H1-1E'

backend = QuantinuumBackend(device_name= machine)
backend.login()

Get the status of the machine

In [4]:
print(machine, "status: ", QuantinuumBackend.device_state(device_name=machine))

H1-1E status:  online


## Circuit Compilation

In [5]:
compiled_circuit = backend.get_compiled_circuit(circuit,optimisation_level=2)
render_circuit_jupyter(compiled_circuit)

## Circuit Cost

In [8]:
n_shots = 1000
backend.cost(compiled_circuit, n_shots=n_shots, syntax_checker='H1-1SC')

11.6

## Run the Circuit

In [9]:
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)
print(handle)

('80c4c17a48544383b26f0ed43cb37901', 'null', 2, '[["c", 0], ["c", 1]]')


In [10]:
import json 
with open('pytket_example_job_handle.json','w') as file:
    json.dump(str(handle), file)

## Check Job Status

In [12]:
status = backend.circuit_status(handle)
print(status)

CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "Bell State Test", "submit-date": "2024-03-08T09:54:22.566947", "result-date": "2024-03-08T09:54:25.713772", "queue-position": null, "cost": "11.6", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


## Retrieve Results

In [13]:
from pytket.backends import ResultHandle

with open('pytket_example_job_handle.json', 'r') as file:
    handle_str = json.load(file)

handle = ResultHandle.from_str(handle_str)

In [14]:
result = backend.get_result(handle)

## Save Results

In [15]:
import json 
with open('pytket_example.json','w') as file:
    json.dump(result.to_dict(), file)

In [16]:
from pytket.backends.backendresult import BackendResult
import json
with open('pytket_example.json') as file:
    data = json.load(file)

result = BackendResult.from_dict(data)

## Analyze Results

In [17]:
print(result.get_counts())
print("-----------------------")
print(result.get_distribution())

Counter({(1, 1): 511, (0, 0): 487, (0, 1): 1, (1, 0): 1})
-----------------------
{(0, 0): 0.487, (0, 1): 0.001, (1, 0): 0.001, (1, 1): 0.511}


# Additional Features

## Batch Submission

In [18]:

machine = 'H1-1E'

backend = QuantinuumBackend(device_name= machine)

print(machine, "status: ", QuantinuumBackend.device_state(device_name=machine))

H1-1E status:  online


In [19]:
machine = 'H1-1E'
n_shots = 1000

max_batch_cost = 1000

backend = QuantinuumBackend(device_name=machine)

In [20]:


batch1 = backend.start_batch(max_batch_cost=max_batch_cost, circuit= compiled_circuit, n_shots=n_shots)

Additional Jobs can be added to the batch using the `add_to_batch` function. The end of a batch can optionally be specified with `batch_end` flag

In [21]:
batch2 = backend.add_to_batch(batch1, compiled_circuit, n_shots=n_shots)
batch3 = backend.add_to_batch(batch1, compiled_circuit, n_shots=n_shots, batch_end=True)

The status for the batch jobs can be checked once submitted

In [26]:
handle_list = [batch1, batch2, batch3]

status_list = [backend.circuit_status(handle) for handle in handle_list]

status_list

[CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "Bell State Test", "submit-date": "2024-03-08T09:54:52.707575", "result-date": "2024-03-08T09:55:00.553796", "queue-position": null, "cost": "11.6", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None),
 CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "Bell State Test", "submit-date": "2024-03-08T09:54:53.958565", "result-date": "2024-03-08T09:55:02.673716", "queue-position": null, "cost": "11.6", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None),
 CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "Bell State Test", "submi

Results for the batch submissions can be returned using `get_results`

In [27]:
result = backend.get_results(handle_list)

result

[BackendResult(q_bits={},c_bits={c[0]: 0, c[1]: 1},counts=None,shots=[[  0]
  [192]
  [  0]
  [192]
  [  0]
  [192]
  [  0]
  [192]
  [  0]
  [  0]
  [192]
  [  0]
  [192]
  [192]
  [192]
  [192]
  [192]
  [  0]
  [192]
  [192]
  [192]
  [  0]
  [192]
  [192]
  [192]
  [192]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [192]
  [192]
  [  0]
  [192]
  [192]
  [192]
  [  0]
  [192]
  [  0]
  [  0]
  [  0]
  [192]
  [  0]
  [192]
  [192]
  [192]
  [  0]
  [192]
  [  0]
  [  0]
  [  0]
  [192]
  [192]
  [  0]
  [192]
  [  0]
  [  0]
  [  0]
  [192]
  [192]
  [192]
  [192]
  [192]
  [192]
  [192]
  [  0]
  [  0]
  [ 64]
  [  0]
  [192]
  [  0]
  [  0]
  [  0]
  [192]
  [  0]
  [192]
  [  0]
  [  0]
  [192]
  [  0]
  [  0]
  [  0]
  [192]
  [192]
  [  0]
  [  0]
  [192]
  [192]
  [192]
  [192]
  [  0]
  [192]
  [  0]
  [192]
  [  0]
  [ 64]
  [  0]
  [192]
  [192]
  [  0]
  [192]
  [192]
  [192]
  [  0]
  [192]
  [  0]
  [192]
  [  0]
  [  0]
  [  0]
  [  0]
  [192]
  [192]
  [192]
  [192]
  [ 

## Parameterized Circuits

In [28]:
from pytket.circuit import fresh_symbol

# set up a parameterised circuit
a = fresh_symbol('a')
circuit = Circuit(3, name="Parameterised Circuit")
circuit.X(0) # Pauli-X gate
circuit.CX(0,1).CX(1,2) # CNOT gate 

circuit.Rz(a, 2) # Rotation-Z gate
circuit.CX(1,2).CX(0,1) # CNOT gate

render_circuit_jupyter(circuit)

Now create a similar circuit, but with values of `a`

In [29]:
simulation_circuit = circuit.copy()
simulation_circuit.symbol_substitution({a: 0.5})
simulation_circuit.measure_all()

render_circuit_jupyter(simulation_circuit)

In [30]:
compiled_circuit = backend.get_compiled_circuit(simulation_circuit,optimisation_level=2)
render_circuit_jupyter(compiled_circuit)

In [31]:
n_shots = 1000
backend.cost(compiled_circuit, n_shots=n_shots, syntax_checker='H1-1SC')

18.4

In [32]:
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)

In [33]:
status = backend.circuit_status(handle)
print(status)

CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "Parameterised Circuit", "submit-date": "2024-03-08T09:56:25.088009", "result-date": "2024-03-08T09:56:27.854289", "queue-position": null, "cost": "18.4", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


In [34]:
result = backend.get_result(handle)
result

BackendResult(q_bits={},c_bits={c[0]: 0, c[1]: 1, c[2]: 2},counts=None,shots=[[128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [  0]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]

## Conditional Gates

In [35]:
from pytket.circuit import Circuit, if_bit

# create a circuit with a classical control

circ = Circuit(name="Conditional Gate")
qreg = circ.add_q_register("q", 3)
creg = circ.add_c_register("c", 2)


circ.X(qreg[0]).H(qreg[0])

# prepare a bell state on qubits 1 and 2
circ.H(qreg[1])
circ.CX(qreg[1], qreg[2])

# construct the teleportatin protocol
circ.CX(qreg[0], qreg[1])
circ.H(qreg[0])
circ.Measure(qreg[0], creg[0])
circ.Measure(qreg[1], creg[1])


# if (creg[1] == 1)
circ.X(qreg[2], condition = if_bit(creg[1]))

# if (creg[0] == 1)
circ.Z(qreg[2], condition = if_bit(creg[0]))

render_circuit_jupyter(circ)